# Lecture 4 - 21.06.2021

# Exercise 1: The eigenvalues of the polarimetric coherency matrix, entropy, anisotropy

* Acquisition: Nkok (Gabon), DLR's F-SAR, L-band

* Path to images: ./data/02-polsar/

* SLC (single-look complex) images:
    * HH: slc_16afrisr0107_Lhh_tcal_test.rat
    * HV: slc_16afrisr0107_Lhv_tcal_test.rat
    * VH: slc_16afrisr0107_Lvh_tcal_test.rat
    * VV: slc_16afrisr0107_Lvh_tcal_test.rat


In [40]:
# import useful libraries, functions, and modules

import sys
sys.path.append('/projects/src/')

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import filters
from ste_io import *

In [41]:
def calculate_correlation(im1, im2, looksr, looksa) : 
    
    corr = filters.uniform_filter(np.real(im1*np.conj(im2)), [looksa,looksr]) + 1j* \
                filters.uniform_filter(np.imag(im1*np.conj(im2)), [looksa,looksr])
    
    return corr

In [42]:
def calculate_eigenvaues_3(T11, T12, T13, T22, T23, T33):

    # Calculate and order (from max to min) the eigenvalues of a 3x3 hermitian matrix in closed-form.
    # Inputs can be 2D rg-az.

    # get dimensions
    dim = T11.shape

    # calculate auxiliary quantities
    A = T11*T22 + T11*T33 + T22*T33 - T12*np.conj(T12) - T13*np.conj(T13) - T23*np.conj(T23)
    B = T11**2 - T11*T22 + T22**2 -T11*T33 -T22*T33 + T33**2 + 3*T12*np.conj(T12) + 3*T13*np.conj(T13) + 3*T23*np.conj(T23)

    DET = T11*T22*T33 - T33*T12*np.conj(T12) - T22*T13*np.conj(T13) - T11*T23*np.conj(T23) + T12*np.conj(T13)*T23 + np.conj(T12)*T13*np.conj(T23)  
    TR = T11 + T22 + T33 
    Z = 27*DET-9*A*TR + 2*TR**3 + np.sqrt((27*DET-9*A*TR + 2*TR**3)**2-4*B**3)
    
    del DET
    
    # ... and here they are:
    LA = ( 1/3.*TR + 2**(1/3.)*B/(3*Z**(1/3.)) + Z**(1/3.)/(3*2**(1/3.)) )
    LB = ( 1/3.*TR - (1+1j*np.sqrt(3))*B/(3*2**(2/3.)*Z**(1/3.)) - (1-1j*np.sqrt(3))*Z**(1/3.)/(6*2**(1/3.)) )
    LC = ( 1/3.*TR - (1-1j*np.sqrt(3))*B/(3*2**(2/3.)*Z**(1/3.)) - (1+1j*np.sqrt(3))*Z**(1/3.)/(6*2**(1/3.)) )
    
    # now order them:
    dumm = np.zeros((dim[0], dim[1], 3), 'float32')
    dumm [:, :, 0] = np.real(LA)
    del LA
    dumm [:, :, 1] = np.real(LB)
    del LB    
    dumm [:, :, 2] = np.real(LC)
    del LC  
    
    L1 = np.max(dumm, axis = 2)
    L3 = np.min(dumm, axis = 2)
    L2 = np.sum(dumm, axis = 2) - L1 - L3
    
    del dumm
    
    return L1, L2, L3
    

In [43]:
def calculate_eigenvectors_3(T11, T12, T13, T22, T23, T33, L1, L2, L3) :

    # Calculate the eigenvectors corresponding to the eigenvalues (L1, L2, L3)
    # of a 3x3 matrix 
    # Inputs can be 2D rg-az.

    # get dimension
    dim = T11.shape    
    
    # now calculate the first eigenvector - corresponds to the maximum eigenvalue L1
    U1 = np.ones((dim[0], dim[1], 3), 'complex64')
    U1[:, :, 0] = (L1 -T33)/np.conj(T13) + (((L1-T33)*np.conj(T12) + np.conj(T13)*T23)*np.conj(T23))/ \
                    (((T22-L1)*np.conj(T13) - np.conj(T12)*np.conj(T23))*np.conj(T13))
    U1[:, :, 1] = -((L1-T33)*np.conj(T12)+np.conj(T13)*T23) / ((T22-L1)*np.conj(T13) - np.conj(T12)*np.conj(T23))
    
    # second eigenvector - corresponds to the eigenvalue L2
    U2 = np.ones((dim[0], dim[1], 3), 'complex64')
    U2[:, :, 0] = (L2 -T33)/np.conj(T13) + (((L2-T33)*np.conj(T12) + np.conj(T13)*T23)*np.conj(T23))/ \
                    (((T22-L2)*np.conj(T13) - np.conj(T12)*np.conj(T23))*np.conj(T13))
    U2[:, :, 1] = -((L2-T33)*np.conj(T12)+np.conj(T13)*T23) / ((T22-L2)*np.conj(T13) - np.conj(T12)*np.conj(T23))
    
    # third eigenvector - corresponds to the minimum eigenvalue L3
    U3 = np.ones((dim[0], dim[1], 3), 'complex64')
    U3[:, :, 0] = (L3 -T33)/np.conj(T13) + (((L3-T33)*np.conj(T12) + np.conj(T13)*T23)*np.conj(T23))/ \
                    (((T22-L3)*np.conj(T13) - np.conj(T12)*np.conj(T23))*np.conj(T13))
    U3[:, :, 1] = -((L3-T33)*np.conj(T12)+np.conj(T13)*T23) / ((T22-L3)*np.conj(T13) - np.conj(T12)*np.conj(T23))   
    
    # and finally normalize to get orthonormal eigenvectors
    norm1 = np.sqrt( np.abs(U1[:,:,0])**2 + np.abs(U1[:,:,1])**2 + np.abs(U1[:,:,2])**2)
    norm2 = np.sqrt( np.abs(U2[:,:,0])**2 + np.abs(U2[:,:,1])**2 + np.abs(U2[:,:,2])**2)    
    norm3 = np.sqrt( np.abs(U3[:,:,0])**2 + np.abs(U3[:,:,1])**2 + np.abs(U3[:,:,2])**2)        
    for nn in range(3):
        U1[:,:,nn] = U1[:,:,nn] / norm1
        U2[:,:,nn] = U2[:,:,nn] / norm2
        U3[:,:,nn] = U3[:,:,nn] / norm3
        
    del norm1
    del norm2
    del norm3     
    
    return U1, U2, U3


**Step 1 : Load data**

In [44]:
# path 2 images
path2i = './data/02-polsar/'
# path2i = '//hr-fs02.intra.dlr.de/Pol-InSAR_InfoRetrieval/01_projects/' +\
#            'HR-FS01-F-SAR-Data/AFRISAR/16AFRISR/FL01/PS07/Tcal_test/RGI/RGI-SR/'

# path 2 outputs
path2o = './Processing_outputs/'

# data set name
dataname = 'L-band'

# looks
looksr = 7
looksa = 7

# open ...
slchh = rrat(path2i + 'slc_16afrisr0107_Lhh_tcal_test.rat', block = [5000, 6000, 0, 2000])
slcvv = rrat(path2i + 'slc_16afrisr0107_Lvv_tcal_test.rat', block = [5000, 6000, 0, 2000])
slchv = rrat(path2i + 'slc_16afrisr0107_Lhv_tcal_test.rat', block = [5000, 6000, 0, 2000])


**Step 2 : Calculate the necessary elements of the coherency matrix**

In [45]:
# Make pauli components
p1 = (slchh + slcvv) # / np.sqrt(2)
p2 = (slchh - slcvv) # / np.sqrt(2)
p3 = 2*slchv # / np.sqrt(2)

del slchh
del slcvv
del slchv

# now calculate correlations
T11 = calculate_correlation(p1, p1, looksr, looksa)

# ... complete with all the rest :-)


del p1, p2, p3


**Step 3 : Calculate eigenvalues**

In [46]:
lambda1, lambda2, lambda3 = calculate_eigenvaues_3(T11, T12, T13, T22, T23, T33)

**Step 4 : Calculate entropy**

In [47]:
# ... simply apply definition ! -> entropy



**Step 5 : Calculate anisotropy** 

In [48]:
# ... apply definition again ! -> anisotropy



**Step 6 : Calculate eigenvectors**

In [ ]:
# ... Use function

U1, U2, U3 = calculate_eigenvectors_3(T11, T12, T13, T22, T23, T33, lambda1, lambda2, lambda3)

del T12, T13, T23

**Step 7 : Calculate mean alpha angle**

In [ ]:
# ... apply definition again! -> alpha (in degrees)




In [ ]:
# save outputs for further comparisons ...

dumm = srat(path2o + 'entropy_' + dataname + '.rat', entropy)
dumm = srat(path2o + 'anisotropy_' + dataname + '.rat', anisotropy)
dumm = srat(path2o + 'alpha_' + dataname + '.rat', alpha)


**Step 8 : Plot**

In [ ]:
# make pauli RGB

dimaz = T11.shape[0]
dimrg = T11.shape[1]
rgb_pauli = np.zeros((dimrg, dimaz, 3), 'float32')

rgb_pauli[:, :, 0] = np.transpose( np.clip(np.sqrt(np.abs(T22)), 0, 2.5*np.mean(np.sqrt(np.abs(T22)))) )     # red
rgb_pauli[:, :, 1] = np.transpose( np.clip(np.sqrt(np.abs(T33)), 0, 2.5*np.mean(np.sqrt(np.abs(T33)))) )    # green
rgb_pauli[:, :, 2] = np.transpose( np.clip(np.sqrt(np.abs(T11)), 0, 2.5*np.mean(np.sqrt(np.abs(T11)))) )    # blue
rgb_pauli[:, :, 0] = rgb_pauli[:, :, 0] / np.max(rgb_pauli[:, :, 0])     # red
rgb_pauli[:, :, 1] = rgb_pauli[:, :, 1] / np.max(rgb_pauli[:, :, 1])     # green
rgb_pauli[:, :, 2] = rgb_pauli[:, :, 2] / np.max(rgb_pauli[:, :, 2])     # blue

del T11, T22, T33

# not start to plot

plt.figure( figsize = (18, 36) )

plt.subplot(7, 1, 1)
plt.imshow(rgb_pauli, aspect = 'auto', interpolation = 'nearest')
plt.title('Pauli')
plt.colorbar()

plt.subplot(7, 1, 2)
plt.imshow(np.transpose(lambda1), aspect = 'auto', vmin = 0, vmax = .5, interpolation = 'nearest', cmap = 'jet')
plt.title('1st eigenvalue')
plt.colorbar()

plt.subplot(7, 1, 3)
plt.imshow(np.transpose(lambda2/lambda1), aspect = 'auto', vmin = 0, vmax=1, interpolation = 'nearest', cmap = 'jet')
plt.title('2nd eigenvalue / 1st eigenvalue')
plt.colorbar()

plt.subplot(7, 1, 4)
plt.imshow(np.transpose(lambda3/lambda1), aspect = 'auto', vmin=0, vmax=1, interpolation = 'nearest', cmap = 'jet')
plt.title('3rd eigenvalue / 1st eigenvalue')
plt.colorbar()

plt.subplot(7, 1, 5)
plt.imshow(np.transpose(entropy), aspect = 'auto', vmin=0, vmax=1, interpolation = 'nearest', cmap = 'jet')
plt.title('Entropy')
plt.colorbar()

plt.subplot(7, 1, 6)
plt.imshow(np.transpose(anisotropy), aspect = 'auto', vmin=0, vmax=1, interpolation = 'nearest', cmap = 'jet')
plt.title('Anisotropy')
plt.colorbar()

plt.subplot(7, 1, 7)
plt.imshow(np.transpose(alpha), aspect = 'auto', vmin=0, vmax=90, interpolation = 'nearest', cmap = 'jet')
plt.title('Alpha')
plt.colorbar()

